In [1]:
data_var = '2023-12-09'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9185,2023-12-09,Brasil Nbb,17:00,Pinheiros,São José,1.95,1.77,151.5,1.88,1.79,-4.5,2.09,1.66,E9sQxDuh,0.512821,0.564972,0.531915,0.558659,0.077792,0.0,0.0,NaN,NaN,180.750,54.343317,0.300655,160.910,87.248074,0.542217,0.000,0.000,93.75,122.36,0.0,0.0,0.0,0.0,0.068430,0.034681,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9186,2023-12-09,Brasil Nbb,17:00,São Paulo,R10 Score Vasco da Gama,1.39,2.84,149.5,1.84,1.86,-6.5,1.95,1.76,06wMwgQo,0.719424,0.352113,0.543478,0.537634,0.071537,0.2,0.0,NaN,NaN,130.106,41.860487,0.321741,130.264,45.551892,0.349689,137.420,0.000,185.50,140.58,0.0,0.0,0.0,0.0,0.484778,0.007644,0.072426,-1.13,-0.226,-1.725664,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9187,2023-12-09,Brasil Nbb,18:00,Bauru,Brasília,1.12,5.61,155.5,1.83,1.87,-13.5,1.97,1.74,nNuYziA4,0.892857,0.178253,0.546448,0.534759,0.071110,0.6,0.4,NaN,NaN,148.160,75.151310,0.507231,293.974,261.628181,0.889970,115.308,379.812,116.56,758.45,0.0,1.0,0.0,0.0,0.943509,0.015289,0.087674,1.94,0.388,0.309278,0.0,0.0,0.0,-2.75,-0.550,-8.381818,0.0,0.0,0.0
9188,2023-12-09,Brasil Nbb,18:00,Franca,Cerrado,1.03,10.10,159.5,1.85,1.85,-18.5,2.00,1.72,8WtUyXfb,0.970874,0.099010,0.540541,0.540541,0.069884,0.6,0.2,NaN,NaN,107.892,14.268872,0.132251,236.320,189.625330,0.802409,103.428,435.988,104.00,567.80,0.0,1.0,0.0,0.0,1.152463,0.000000,0.106446,-0.63,-0.126,-0.238095,0.0,0.0,0.0,3.35,0.670,13.582090,0.0,0.0,0.0
9189,2023-12-09,Brasil Nbb,19:00,Corinthians Paulista,Botafogo,1.21,4.12,156.5,1.87,1.79,-10.5,1.98,1.74,4bzwzBPA,0.826446,0.242718,0.534759,0.558659,0.069165,0.0,0.8,NaN,NaN,141.944,37.392544,0.263432,328.106,136.740151,0.416756,0.000,454.480,93.44,365.75,0.0,0.0,0.0,0.0,0.772113,0.030912,0.091240,0.00,0.000,inf,0.0,0.0,0.0,-5.00,-1.000,-3.120000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9516,2023-12-09,Eua Ncaa,23:00,Utah Tech,Idaho,1.30,3.66,141.5,1.91,1.91,-8.5,2.00,1.71,zR4yZ33r,0.769231,0.273224,0.523560,0.523560,0.042455,0.0,0.0,NaN,NaN,251.020,152.562896,0.607772,457.428,440.268606,0.962487,0.000,0.000,0.00,577.50,0.0,1.0,0.0,0.0,0.672892,0.000000,0.110545,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9517,2023-12-09,Eua Ncaa,23:00,Weber State,Cal Poly,1.09,8.20,128.5,1.80,1.95,-14.5,2.00,1.71,Kt7uR95Q,0.917431,0.121951,0.555556,0.512821,0.039382,0.0,0.6,NaN,NaN,236.628,247.822660,1.047309,323.488,147.346645,0.455493,0.000,393.034,213.75,516.60,0.0,1.0,0.0,0.0,1.082353,0.056569,0.110545,0.00,0.000,inf,0.0,0.0,0.0,-1.43,-0.286,-25.174825,0.0,0.0,0.0
9518,2023-12-09,Europa Liga Bnxt,15:30,Groningen,Den Helder,1.03,10.10,144.5,1.84,1.86,-20.5,2.06,1.68,xdrW7WWa,0.970874,0.099010,0.543478,0.537634,0.069884,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,64.98,240.04,0.0,0.0,1.0,0.0,1.152463,0.007644,0.143690,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9519,2023-12-09,Macedônia Do Norte Prva Liga,14:00,Ohrid,Cair 2030,11.50,1.02,164.5,1.83,1.85,17.5,2.02,1.71,8WMziSZg,0.086957,0.980392,0.546448,0.540541,0.067349,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,490.70,267.38,1.0,0.0,0.0,0.0,1.183783,0.007686,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
9,00:00,Eua Nba,Portland Trail Blazers,Dallas Mavericks,230.5,1.82,1.0000,Over
10,00:00,Eua Nba,Utah Jazz,Los Angeles Clippers,223.5,1.86,1.0000,Over
11,22:30,Eua Torneio Nba In,Los Angeles Lakers,Indiana Pacers,238.5,1.83,1.0000,Over
89,14:00,Eua Ncaa,East Carolina,South Carolina,140.5,1.91,0.8484,Over
92,14:00,Eua Ncaa,Tennessee,Illinois,145.5,1.91,0.8675,Over
95,15:00,Eua Ncaa,VMI,American,146.5,1.91,0.8523,Over
96,15:00,Eua Ncaa,Western Michigan,Youngstown State,144.5,1.91,0.8580,Over
97,15:30,Eua Ncaa,LSU,Kansas State,144.5,1.91,0.8706,Over
100,16:00,Eua Ncaa,Davidson,Miami (Ohio),141.5,1.91,0.8677,Over
104,16:15,Eua Ncaa,Duke,Charlotte,137.5,1.91,0.8877,Over
